In [6]:
!pip install arguebuf


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# change the node type to edges

# Function to modify the graph
def modify_graph(graph):
    for node in graph.nodes:
        if graph.nodes[node]['label'] == 'Support':
            predecessors = list(graph.predecessors(node))
            successors = list(graph.successors(node))

            if len(predecessors) == 1 and len(successors) == 1:
                # Get the predecessor and successor nodes
                pred_node = predecessors[0]
                succ_node = successors[0]

                # Remove the 'support' node and create a direct edge
                graph.remove_edge(pred_node, node)
                graph.remove_edge(node, succ_node)
                graph.add_edge(pred_node, succ_node, type='support')

        if graph.nodes[node]['label'] == 'Attack':
            predecessors = list(graph.predecessors(node))
            successors = list(graph.successors(node))

            if len(predecessors) == 1 and len(successors) == 1:
                # Get the predecessor and successor nodes
                pred_node = predecessors[0]
                succ_node = successors[0]

                # Remove the 'attack' node and create a direct edge
                graph.remove_edge(pred_node, node)
                graph.remove_edge(node, succ_node)
                graph.add_edge(pred_node, succ_node, type='attack')

        if graph.nodes[node]['label'] == 'Rephrase':
            predecessors = list(graph.predecessors(node))
            successors = list(graph.successors(node))

            if len(predecessors) == 1 and len(successors) == 1:
                # Get the predecessor and successor nodes
                pred_node = predecessors[0]
                succ_node = successors[0]

                # Remove the 'attack' node and create a direct edge
                graph.remove_edge(pred_node, node)
                graph.remove_edge(node, succ_node)
                graph.add_edge(pred_node, succ_node, type='rephrase')

def remove_support_attack_nodes(graph):
    nodes_to_remove = []

    for node in graph.nodes:
        if graph.nodes[node]['label'] == 'Support' or graph.nodes[node]['label'] == 'Attack' or graph.nodes[node]['label'] == 'Rephrase':
            nodes_to_remove.append(node)

    graph.remove_nodes_from(nodes_to_remove)

In [7]:
# Function to process children nodes
def process_nodes(graph, node, previous_summary=None):
        claim = graph.nodes[node]['label']
        children = list(graph.predecessors(node))

        support_premise = []
        attack_premise = []

        # Group premises based on support or attack
        for child in children:
            edge_type = graph.edges[child, node]['type']
            if edge_type == 'support':
                support_premise.append(graph.nodes[child]['label'])
            elif edge_type == 'attack':
                attack_premise.append(graph.nodes[child]['label'])

        # Generate text based on grouped premises using BART
        support_text = " ".join(support_premise)
        attack_text = " ".join(attack_premise)

        if support_text and attack_text:
            text_input = f'{claim} because, {support_text}. However,{attack_text}'
        elif support_text:
            text_input = f'{claim} because, {support_text}'
        elif attack_text:
            text_input = f'{claim} but, {attack_text}'


        if previous_summary:
            text_input = f"{previous_summary}{text_input}"

        print('text_input',text_input)
        summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        summary = summarizer(text_input, max_length=100, min_length=80)
        print('summary',summary)

        summary_text = summary[0]['summary_text']
        # Get the next level of nodes
        next_nodes = [child for child in children if graph.in_degree(child) > 0]
        for n in next_nodes:
          summary_text = process_nodes(graph, n, summary_text)

        return summary_text

In [8]:
import os
import arguebuf as ag
import networkx as nx
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

# function to porcess arg graph and create bart divide conquer summaries
def create_bart_divideconquer_summaries(input_folder, output_folder):

    # Iterate through files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            input_file_path = os.path.join(input_folder, filename)
            graph = ag.load.file(input_file_path)
            ngraph = ag.dump.networkx(graph)

            # Call the function to modify the graph
            modify_graph(ngraph)
            # Call the function to remove support and attack nodes
            remove_support_attack_nodes(ngraph)

            # Start processing from the root node (claim)
            source_node = [node for node in ngraph.nodes() if len(list(ngraph.successors(node))) == 0][0]
            final_summary = process_nodes(ngraph, source_node)

            # write the concatenated result to a text file in the output folder
            output_file_path = os.path.join(output_folder, filename)
            with open(output_file_path, 'w', encoding="utf8") as output_file:
                output_file.write(final_summary)

In [9]:
# input and output folder paths
# input - arg graphs
# output - bart summaries - using divide and conquer.
# Replace the below with desired input and output path

#input_folder_path = 'C:/Users/Nila/Documents/Main Scripts_25.12.2023/Jupyter Notebooks/complete testing/llm_input/arg_graphs/'
#output_folder_path = 'C:/Users/Nila/Documents/Main Scripts_25.12.2023/Jupyter Notebooks/complete testing/llm_output/BART/divide_conquer'

input_folder_path = "C:/Users/Nila/Documents/Project/benchmark/data/kialo-nilesc/sample-test/input/"
output_folder_path = "C:/Users/Nila/Documents/Project/benchmark/data/kialo-nilesc/sample-test/output/"

create_bart_divideconquer_summaries(input_folder_path, output_folder_path)

text_input Shame plays an important role in society. because, Keeps people conforming. Ensures people does the right thing Ensures religions are maintained Provides a useful tool to teach people right from wrong. However,A form of control that suppresses individuality Creates feelings of guilt Can create mental health problems Culturally constructed and meaningless


Your max_length is set to 150, but your input_length is only 66. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=33)


summary [{'summary_text': "Shame plays an important role in society. because, Keeps people conforming. Ensures people does the right thing Ensures religions are maintained. Provides a useful tool to teach people right from wrong. However, a form of control that suppresses individuality Creates feelings of guilt Can create mental health problems Culturally constructed and meaningless. The word 'shame' can be used to refer to any form of negative self-consciousness or self-loathing."}]
text_input Shame plays an important role in society. because, Keeps people conforming. Ensures people does the right thing Ensures religions are maintained. Provides a useful tool to teach people right from wrong. However, a form of control that suppresses individuality Creates feelings of guilt Can create mental health problems Culturally constructed and meaningless. The word 'shame' can be used to refer to any form of negative self-consciousness or self-loathing.Keeps people conforming. because, Shame is

Your max_length is set to 150, but your input_length is only 109. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=54)


summary [{'summary_text': "'Shame' can be used to refer to any form of negative self-consciousness or self-loathing. Shame plays an important role in society. because, Keeps people conforming. Ensures people does the right thing Ensures religions are maintained. Provides a useful tool to teach people right from wrong. However, a form of control that suppresses individuality Creates feelings of guilt Can create mental health problems Culturally constructed and meaningless."}]
text_input 'Shame' can be used to refer to any form of negative self-consciousness or self-loathing. Shame plays an important role in society. because, Keeps people conforming. Ensures people does the right thing Ensures religions are maintained. Provides a useful tool to teach people right from wrong. However, a form of control that suppresses individuality Creates feelings of guilt Can create mental health problems Culturally constructed and meaningless.Shame is one of the biggest fears because, This is especiall

...

summary [{'summary_text': "'Shame' can be used to refer to any form of negative self-consciousness or self-loathing. Shame plays an important role in society. because, Keeps people conforming. Ensures people does the right thing. Provides a useful tool to teach people right from wrong. However, a form of control that suppresses individuality Creates feelings of guilt Can create mental health problems Culturally constructed and meaningless. This is especially true for women."}]
text_input 'Shame' can be used to refer to any form of negative self-consciousness or self-loathing. Shame plays an important role in society. because, Keeps people conforming. Ensures people does the right thing. Provides a useful tool to teach people right from wrong. However, a form of control that suppresses individuality Creates feelings of guilt Can create mental health problems Culturally constructed and meaningless. This is especially true for women.Creates feelings of guilt because, People are unlikely t

Your max_length is set to 150, but your input_length is only 130. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=65)


summary [{'summary_text': "'Shame' can be used to refer to any form of negative self-consciousness or self-loathing. Shame plays an important role in society. because, Keeps people conforming. Ensures people does the right thing. Provides a useful tool to teach people right from wrong. However, a form of control that suppresses individuality Creates feelings of guilt Can create mental health problems Culturally constructed and meaningless. This is especially true for women."}]
text_input 'Shame' can be used to refer to any form of negative self-consciousness or self-loathing. Shame plays an important role in society. because, Keeps people conforming. Ensures people does the right thing. Provides a useful tool to teach people right from wrong. However, a form of control that suppresses individuality Creates feelings of guilt Can create mental health problems Culturally constructed and meaningless. This is especially true for women.Provides a useful tool to teach people right from wrong 

Your max_length is set to 150, but your input_length is only 128. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)


summary [{'summary_text': "'Shame' can be used to refer to any form of negative self-consciousness or self-loathing. Shame plays an important role in society. because, Keeps people conforming. Ensures people does the right thing. Provides a useful tool to teach people right from wrong. However, a form of control that suppresses individuality Creates feelings of guilt Can create mental health problems Culturally constructed and meaningless. This is especially true for women."}]


In [ ]:
# step by step results genererated for title
# 'A bar of soap is better than a bottle of shower gel'

"""
text_input: A bar of soap is better than a bottle of shower gel. because, Shower gels generate a lot more plastic packaging which isn’t always responsibly recycled Shower gel has a lot more bulk and water weight so is inefficient to transport/distribute Soap is more convenient in use (no bottle to unscrew/open A bar of soap lasts longer. A bar of soap uses less packaging than shower gel Soap is cheaper than shower gel. However,Shower gel is more hygienic, especially for a shared shower It is difficult to wash your hair with a bar of soap. Soap stings your eyes Soap is old-fashioned It is easier to drop soap in the shower.
summary: [{'summary_text': 'A bar of soap is better than a bottle of shower gel because, Shower gels generate a lot more plastic packaging which isn’t always responsibly recycled. Soap is more convenient in use (no bottle to unscrew/open) Soap lasts longer and is cheaper than shower gel. However,Shower gel is more hygienic, especially for a shared shower. It is easier to drop soap in the shower.'}]
text_input: A bar of soap is better than a bottle of shower gel because, Shower gels generate a lot more plastic packaging which isn’t always responsibly recycled. Soap is more convenient in use (no bottle to unscrew/open) Soap lasts longer and is cheaper than shower gel. However,Shower gel is more hygienic, especially for a shared shower. It is easier to drop soap in the shower.. Shower gel is more hygienic, especially for a shared shower because, After one person used a bar of soap there might be hair sticking to it.
summary: [{'summary_text': 'A bar of soap is better than a bottle of shower gel because, Shower gels generate a lot more plastic packaging which isn’t always responsibly recycled. Soap is more convenient in use (no bottle to unscrew/open) Soap lasts longer and is cheaper than shower gel. However,Shower gel is more hygienic, especially for a shared shower. It is easier to drop soap in the shower..'}]
text_input: A bar of soap is better than a bottle of shower gel because, Shower gels generate a lot more plastic packaging which isn’t always responsibly recycled. Soap is more convenient in use (no bottle to unscrew/open) Soap lasts longer and is cheaper than shower gel. However,Shower gel is more hygienic, especially for a shared shower. It is easier to drop soap in the shower... It is difficult to wash your hair with a bar of soap. but, Short hair can be just as easily be washed with a soap by just rubbing it over your hair.
summary: [{'summary_text': "A bar of soap is better than a bottle of shower gel because, Shower gels generate a lot more plastic packaging which isn't always responsibly recycled. Soap is more convenient in use (no bottle to unscrew/open) Soap lasts longer and is cheaper than shower gel. However,Shower gel is more hygienic, especially for a shared shower. It is easier to drop soap in the shower... It is difficult to wash your hair with a soap."}]
text_input: A bar of soap is better than a bottle of shower gel because, Shower gels generate a lot more plastic packaging which isn't always responsibly recycled. Soap is more convenient in use (no bottle to unscrew/open) Soap lasts longer and is cheaper than shower gel. However,Shower gel is more hygienic, especially for a shared shower. It is easier to drop soap in the shower... It is difficult to wash your hair with a soap.. Soap stings your eyes but, A lot of soap does not sting in the eyes. Shower gel can sting in your eyes too.
summary: [{'summary_text': 'A bar of soap is better than a bottle of shower gel because, Shower gels generate a lot more plastic packaging. Soap is more convenient in use (no bottle to unscrew/open) Soap lasts longer and is cheaper than shower gel. However,Shower gel is more hygienic, especially for a shared shower. It is easier to drop soap in the shower... It is difficult to wash your hair with a soap.. Soap stings your eyes but, A lot of soap does not sting in the eyes.'}]
text_input: A bar of soap is better than a bottle of shower gel because, Shower gels generate a lot more plastic packaging. Soap is more convenient in use (no bottle to unscrew/open) Soap lasts longer and is cheaper than shower gel. However,Shower gel is more hygienic, especially for a shared shower. It is easier to drop soap in the shower... It is difficult to wash your hair with a soap.. Soap stings your eyes but, A lot of soap does not sting in the eyes.. A bar of soap uses less packaging than shower gel because, Given the immense damage plastic does to the environment, this is a vital factor to consider.
summary: [{'summary_text': 'A bar of soap is better than a bottle of shower gel because, Shower gels generate a lot more plastic packaging. Soap is more convenient in use (no bottle to unscrew/open) Soap lasts longer and is cheaper than shower gel. However,Shower gel is more hygienic, especially for a shared shower. It is easier to drop soap in the shower... It is difficult to wash your hair with a soap.. Soap stings your eyes but, A lot of soap does not sting in the eyes..'}]
final_summary A bar of soap is better than a bottle of shower gel because, Shower gels generate a lot more plastic packaging. Soap is more convenient in use (no bottle to unscrew/open) Soap lasts longer and is cheaper than shower gel. However,Shower gel is more hygienic, especially for a shared shower. It is easier to drop soap in the shower... It is difficult to wash your hair with a soap.. Soap stings your eyes but, A lot of soap does not sting in the eyes..
"""